In [1]:
import dask
import dask.dataframe as dd
from dask.distributed import Client
import json
import time

In [2]:
client = Client(memory_limit='4GB')
client.restart()

/Users/isaaczeng/miniconda/envs/Course_UCSD_38/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52300 instead
  warnings.warn(
2023-10-11 20:19:13,404 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/93/jk9n3xzx65q49d8f_wg9lrzc0000gn/T/dask-worker-space/worker-gf1_ec3w', purging
2023-10-11 20:19:13,793 - distributed.nanny - WARNING - Restarting worker
2023-10-11 20:19:13,803 - distributed.nanny - WARNING - Restarting worker
2023-10-11 20:19:13,811 - distributed.nanny - WARNING - Restarting worker
2023-10-11 20:19:13,813 - distributed.nanny - WARNING - Restarting worker
2023-10-11 20:19:13,814 - distributed.nanny - WARNING - Restarting worker


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52300/status,
Dashboard: http://127.0.0.1:52300/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52301,Workers: 5
Dashboard: http://127.0.0.1:52300/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:52341,Total threads: 2
Dashboard: http://127.0.0.1:52343/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:52304,


In [3]:
data = dd.read_csv('user_reviews.csv')

In [4]:
helpful_df = data['helpful'].str.extract(r'\[(\d+), (\d+)\]').astype(int)
data['helpful_vote'], data['total_vote'] = helpful_df[0], helpful_df[1]

data['reviewTime'] = data['reviewTime'].astype(str)
data['Year'] = data['reviewTime'].str.split(', ').str.get(1).str.strip().astype(int)

group_data = data.groupby('reviewerID')

merge_df = dd.concat([
    group_data.size(split_out=10),
    group_data['overall'].mean(split_out=10)
], axis=1)

merge_df = dd.concat([
    merge_df,
    group_data['Year'].min(split_out=10),
], axis=1)

merge_df = dd.concat([
    merge_df,
    group_data['helpful_vote'].sum(split_out=10),
], axis=1)

merge_df = dd.concat([
    merge_df,
    group_data['total_vote'].sum(split_out=10)
], axis=1)

merge_df.columns = ['number_products_rated', 'avg_ratings', 'reviewing_since', 'helpful_votes', 'total_votes']

/Users/isaaczeng/miniconda/envs/Course_UCSD_38/lib/python3.8/site-packages/dask/dataframe/groupby.py:1544: FutureWarning: In the future, `sort` for groupby operations will default to `True` to match the behavior of pandas. However, `sort=True` does not work with `split_out>1`. To retain the current behavior for multiple output partitions, set `sort=False`.
  warnings.warn(SORT_SPLIT_OUT_WARNING, FutureWarning)
/Users/isaaczeng/miniconda/envs/Course_UCSD_38/lib/python3.8/site-packages/dask/dataframe/groupby.py:1544: FutureWarning: In the future, `sort` for groupby operations will default to `True` to match the behavior of pandas. However, `sort=True` does not work with `split_out>1`. To retain the current behavior for multiple output partitions, set `sort=False`.
  warnings.warn(SORT_SPLIT_OUT_WARNING, FutureWarning)
/Users/isaaczeng/miniconda/envs/Course_UCSD_38/lib/python3.8/site-packages/dask/dataframe/groupby.py:1544: FutureWarning: In the future, `sort` for groupby operations will 

In [ ]:
start = time.time()
submit = merge_df.describe().compute().round(2)
end = time.time()
print(end - start)

In [18]:
with open('results_PA0.json', 'w') as outfile: 
    json.dump(json.loads(submit.to_json()), outfile)